# ETL – Consumo Mensal de Energia

Este notebook demonstra um pipeline ETL (Extract, Transform, Load) simulando
uma concessionária de energia elétrica que processa medições mensais de consumo
para faturamento e análise.

## 1) Importar pandas

In [ ]:
import pandas as pd

## 2) Extract

Os dados de consumo mensal são fornecidos pela concessionária em formato CSV e lidos pelo pipeline ETL utilizando pandas.

In [ ]:
df_raw = pd.read_csv("../data/raw/consumo_mensal.csv")

## 3) Transform

O consumo mensal é calculado e classificado por meio de uma função.


In [ ]:
df = df_raw.copy()
df["mes"] = pd.to_datetime(df["mes"])

df["valor_conta"] = df["kwh_consumido"] * df["tarifa"]

def classificar_consumo (kwh):
    if kwh <= 150:
        return "Baixo"
    elif kwh <= 300:
        return "Médio"
    else:
        return "Alto"

df["nivel_consumo"] = df["kwh_consumido"].apply(classificar_consumo)
df

## 4) Load

Os dados tratados são salvos em arquivos

In [ ]:
df.to_csv("../data/processado/consumo_tratado.csv", index=False)

In [ ]:
summary = (
    df.groupby("id_cliente")
    .agg(
        kwh_consumido=("kwh_consumido", "mean"),
        valor_pagar=("valor_conta", "mean")
    )
    .reset_index()
)

# Formatar valor em reais

summary["valor_conta-formatado"] = summary["valor_pagar"].apply(
    lambda x: f"R$ {x:.2f}"
)

summary.to_csv("../data/saida/resumo_clientes.csv", index=False)
df.head(), summary